迁移学习模型添加新层
通常在迁移学习中，都是直接将最后一层的全连接层大小换成自己数据集类的大小，  
然后finetune，在这次比赛中，我发现这样的精度并不能提升到最大，因此，采用迁移的base模型来叠加MLP的获取更高的精度，如何在base模型之后，叠加MLP？代码如下：


In [ ]:

class model_bn(nn.Module):
    def __init__(self, model, feature_size):

        super(model_bn, self).__init__() 
        self.features = nn.Sequential(*list(model.children())[:-1])
        self.num_ftrs = model.fc.in_features
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.num_ftrs),
            nn.Dropout(0.5),
            nn.Linear(self.num_ftrs, feature_size),
            nn.BatchNorm1d(feature_size),
            nn.ELU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(feature_size, classes_num),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

在上述代码中，self.features = nn.Sequential(*list(model.children())[:-1]) 直接将全base模型中的最后一层全连接层去除，*是解包的操作，将list类型以元组的方式传递给nn.Sequential对象，之后添加dropout和bn层来获取更高的精度，这里还要注意view函数相当于numpy中的reshape函数，在这里的作用就是keras中的Flatten层，将输出从二维压成一维，在这过程中发现，貌似pytorch没有直接的全局池化层，需要自己定义操作，可以通过以下方式来进行：


In [ ]:
import torch.nn.functional as F
output = F.average_pool2d(input, kernel_size=input.size()[2:])
